In [1]:
%load_ext autoreload
%autoreload 2
import gtfs_utils as gtfs_utils
from gtfs_utils.filter import BoundsFilter

import folium
import folium.plugins
import shapely

In [2]:
gtfs = gtfs_utils.load_gtfs("vienna.gtfs.zip", lazy=False)
stop_gdf = gtfs.stop_gdf(additional_columns=["stop_name", "stop_id"])
stop_gdf.tail()

,stop_name,stop_id,geometry
4536,Paulinensteig,at:49:997:0:1,POINT (16.29908 48.21872)
4537,Paulinensteig,at:49:997:0:2,POINT (16.29931 48.21844)
4538,Pellmanngasse,at:49:998:0:1,POINT (16.28811 48.13086)
4539,Pellmanngasse,at:49:998:0:2,POINT (16.28878 48.13066)
4540,Marco-Polo-Promenade,at:49:999:0:1,POINT (16.42308 48.27743)


# Prepare test for Bounds south of Vienna

In [3]:
vienna_south_bounds = [
    16.2,
    47.95,
    16.35,
    48.1,
]
bounds = shapely.geometry.box(*vienna_south_bounds)

In [4]:
manual_inside = stop_gdf[stop_gdf.within(bounds)]
manual_outside = stop_gdf[~stop_gdf.within(bounds)]

In [5]:
%%time
gtfs = gtfs_utils.load_gtfs("vienna.gtfs.zip", lazy=False)
filtered_gtfs_normal = gtfs_utils.do_filter(
    gtfs, [BoundsFilter(bounds=vienna_south_bounds, complete_trips=False)]
)
stops_gdf_within = filtered_gtfs_normal.stop_gdf("all")

CPU times: user 4.55 s, sys: 375 ms, total: 4.92 s
Wall time: 4.95 s


In [6]:
%%time
gtfs = gtfs_utils.load_gtfs("vienna.gtfs.zip", lazy=False)
filtered_gtfs_complete_trips = gtfs_utils.do_filter(
    gtfs, [BoundsFilter(bounds=vienna_south_bounds, complete_trips=True)]
)
stops_gdf_complete_trips = filtered_gtfs_complete_trips.stop_gdf("all")

CPU times: user 4.64 s, sys: 374 ms, total: 5.02 s
Wall time: 5.06 s


## Unedited gtfs + bounds

In [7]:
m = folium.Map(tiles="Cartodb Positron")
gdf = manual_outside.explore(m=m)
manual_inside.explore(m=m, color="red")
m.fit_bounds(gdf.get_bounds())
folium.GeoJson(bounds, style_function=lambda f: {"color": "red"}).add_to(m)
m

## Filtered Gtfs

In [12]:
m = folium.plugins.DualMap(tiles="Cartodb Positron")
stops_gdf_within.explore(m=m.m1)
gdf = stops_gdf_complete_trips.explore(m=m.m2)
folium.GeoJson(bounds, style_function=lambda f: {"color": "red"}).add_to(m)
m.fit_bounds(gdf.get_bounds())
m

In [9]:
print(len(stop_gdf), "All stops")
print(len(stops_gdf_within), "Within bounds (no complete-trips)")
print(len(stops_gdf_complete_trips), "Within bounds (complete-trips)")
print("---")
print(len(manual_inside), "Within bounds (Manually computed)")
print(len(manual_outside), "Outside bounds (Manually computed)")

4541 All stops
30 Within bounds (no complete-trips)
75 Within bounds (complete-trips)
---
30 Within bounds (Manually computed)
4511 Outside bounds (Manually computed)
